In [3]:
""" It is recommmended NOT to change codes in this cell """
try:
    import google.colab
    !pip install bitsandbytes==0.43.0
    !pip install datasets==2.10.1
    !pip install transformers==4.38.2
    !pip install peft==0.11.0
    !pip install sentencepiece==0.1.99
    !pip install -U accelerate==0.28.0
    !pip install colorama==0.4.6
    !pip install git+https://github.com/bhbbbbb/f-final
except:
    pass

ho colab


In [1]:
import warnings
warnings.filterwarnings("ignore")
from transformers import GenerationConfig
import final
from final.llm import LLM
final.data.set_password(...)

## Instruction & Inference Dataset

It's recommended not to modify this block, but one can play around and check what is used in the inference_dataset.


In [2]:
N_TAGS_PER_PRODUCT = 7
INSTRUCTION = (
    'Predict the next product tags based on browsing histories. '
    f'Provide exactly {N_TAGS_PER_PRODUCT} output tags, comma-separated, sorted by relevance. '
)
PRODUCT_RAW_TAGS = final.data.product_tags_v5_en()

inference_dataset = final.InferenceDataset(
    raw_tags=PRODUCT_RAW_TAGS,
    # user_names=final.data.default_user_names(),
    n_tags_per_product=N_TAGS_PER_PRODUCT,
    legacy_output_first_time_buyer=False,
)
print(PRODUCT_RAW_TAGS)
print(INSTRUCTION)
print(inference_dataset[0])
print(inference_dataset[1])
print(inference_dataset[3])
print(inference_dataset[5])

{5: ['Love Ataulfo Mango', 'Mango', 'Mango Ice', 'Fresh Mangoes', 'Exquisite Fruits', 'Fruit Gift Box', 'Juicy Mango', 'Premium Mango', 'Summer Delight', 'Perfect Gift'], 23: ['Frozen Broccoli', 'Cruciferous Vegetables', 'Cancer-fighting', 'Low-Calorie Diet', 'Healthy Eating', 'Quick Meal Prep', 'Fitness Recipes', 'Weight Loss', 'Low-Carb Option', 'Salad Ingredient'], 24: ['Frozen Cauliflower', 'Healthy Eating', 'Meal Preparation', 'Fitness', 'Weight Control', 'Quick Meals', 'Frozen Vegetables', 'Nutritious Ingredients', 'Diet Control', "Kids' Lunch"], 25: ['Healthy Eating', 'Frozen Vegetables', 'Meal Prep', 'Fitness Food', 'Health Recipes', 'Controlled Diet', 'Quick Meals', 'Broccoli', 'Cauliflower', 'Convenient'], 28: ['Pork Balls', 'Taiwan Pork', 'Hot Pot Ingredients', 'Fresh Meat', 'Pork Leg', 'Meatballs', 'Pure Pork', 'Yummy Soup', 'Local Ingredients', 'Meat Density'], 46: ['Gourmet Fish Dish', 'Taiwan Sea Bass', 'Post-surgery Diet', 'Meal Prep', 'Sea Bass Recipe', 'Nutritious Mea

## LLM Preparation
Download and initialize the corresponding LLM and checkpoint (Both need to be installed).

In [ ]:
# model_name = "/content/taide_7b"
# !wget -O taide_7b.zip "https://www.dropbox.com/scl/fi/rlzdoidgejt9wiox88gz5/taide_7b.zip?rlkey=vtuc8xq7mq2d0dpuijxlb3jfj&dl=0"
# # !gdown 1OiO5N9-wXDPX9XuGnPP-cgOoyFy4p3FD # 如果上面的連結失效（無法下載模型），可以將這行最前面的 "#" 刪除，即可正常下載
# !unzip taide_7b.zip

In [ ]:
# https://huggingface.co/docs/hub/en/security-tokens
# from huggingface_hub import notebook_login
# notebook_login()
# model_name = 'meta-llama/Llama-2-7b-hf'

In [ ]:
# ckpt_name = '/content/ckpt-taide-2'
# !wget -O "ckpt-taide-2.zip" "https://www.dropbox.com/scl/fi/m6ochvs7r2csh4o47ffhh/ckpt-taide-2.zip?rlkey=o5yw0pbo47rsgp0ul62hfq207&st=cgxsvv85&dl=0"
# !unzip "ckpt-taide-2.zip"

In [3]:
model_name = 'togethercomputer/Llama-2-7B-32K-Instruct'
# ckpt_name = 'ckpts/ckpt-2'
llm = LLM(
    model_name=model_name,
    instruction=INSTRUCTION,
    # ckpt_name=ckpt_name,
    real_ckpt_dir='ckpts'
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.27s/it]


loading checkpoint from ckpts/ckpt-14


## Generation

There are three available generation methods.
It's recommended to try single_inference and multi_beam_inference first.

### single_inference

Just inference the model once.

### iterative_inference.

Need to specify `n` (`n >= 2`). This method used the output of $i$-th iteration as the input of $i+1$-th iteration.

### multi_beam_inference

Similar to single inference, but use multiple generation configs. 


In [6]:
"""Single Inference"""

max_len = 128 # not sure
generation_config = GenerationConfig(
    do_sample=True,
    temperature=0.1, # important
    num_beams=1,
    top_p=0.5, # not sure
    no_repeat_ngram_size=3, # important (higher less diversity)
    pad_token_id=2,
)
single_inference_results = {}
for i, pid in enumerate(final.AVAILABLE_PRODUCT_IDS):
    d = inference_dataset[i]
    predict = llm.single_inference(d, generation_config, max_len, verbose=False)
    inputs_ = '\n'.join(d['input'].split('\n')[:-1])
    print(f'inputs{i}:\n{d["instruction"]}\n{inputs_}。\n')
    print(f'outputs:\n' + predict)
    print('-' * 50)
    single_inference_results[pid] = predict

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


inputs0:

1. Love Ataulfo Mango, Mango, Mango Ice, Fresh Mangoes, Exquisite Fruits, Fruit Gift Box, Juicy Mango。

outputs:

1. Love Ataulfo Mango, Mango, Mango Ice, Fresh Mangoes, Exquisite Fruits, Fruit Gift Box, Juicy Mango
2.

 Taiwanese Freshly Harvested Water Peaches, Tropical Ground Peaches and Fresh Frozen Golden Turf, Juice Delicious and Sweet Fruit, Limited-Time Offer, Sweet and Flavorful, Juiciest and Scentious
3. Taiwan Water Honey Peach, Flavored Water, Premium Quality, Juiced Fruit. Taiwanese peaches, Squid, Snardles, Crab, Fruitarian, Seafood delicacy
4. Taiwan Fruit Shrimp, Tropic Fruit Juice,
--------------------------------------------------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


inputs1:

1. Frozen Broccoli, Cruciferous Vegetables, Cancer-fighting, Low-Calorie Diet, Healthy Eating, Quick Meal Prep, Fitness Recipes。

outputs:

1. Frozen Broccoli, Cruciferous Vegetables, Cancer-fighting, Low-Calorie Diet, Healthy Eating, Quick Meal Prep, Fitness Recipes
2.

 Frozen spinach, Healthyman, Convenience, Frozen vegetables, Fresh spinach
3. Frozed Brocc, Crucc, Canceraffecting, Meal Planning, Healthymonging, Frozerbags
4. Frozoned spinach and chicken, Health-conscious freezing, Conventional frozen ingredients, Frozeable vegetables and meat, Health supplements, Wont-flavor-your-food
5. Froze Brocc and Cheese, Quick meal prep, Health food, Froce recipes
--------------------------------------------------


...

inputs353:

1. Hotpot beef slices, BBQ beef slices, Wagyu beef, Marble beef, Grilled beef slices, Premium beef, Australian Wagyu。

outputs:

1. Hotpot beef slices, BBQ beef slices, Wagyu beef, Marble beef, Grilled beef slices, Premium beef, Australian Wagyu
2.
, Akaushi Beef, Prime Grade, American Wagy, BBq meat slices and Grilled Beef Slices, Marbling Texture, Premiums Beef Cuts, Tender Shoulder Beef
3. Beef sells, Hot pot meat, American Blue Ribbon beef cows, Taro beef stew, PremUI beef cubes, Japanese-style beef burgers, Sweet potato beef
4. Beeftrail, Hotpotbeefslices, AmericanBlueTenderedbeef, TarnishedBeef, DryA
--------------------------------------------------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


inputs354:

1. Gourmet Instant Noodles, Premium Udon Noodles, Authentic Ramen, Frozen Ready-to-Eat Noodles, Fine Oolong Noodles, High-Quality Ingredients, Original Flavor Noodles。

outputs:

1. Gourmet Instant Noodles, Premium Udon Noodles, Authentic Ramen, Frozen Ready-to-Eat Noodles, Fine Oolong Noodles, High-Quality Ingredients, Original Flavor Noodles
2.
sweet dessert, Chinese dessert & fruit juice, Silver ear fungus, Nutritious dessert soup, Healthy dessert juice
3. Sweet Potato Delights, Roasted Sweet Pomato, Healthymaking, Organic Farming, Low Carb Diet, Fitness Recipes
4. Thin Dumpling Wrappers, Non-GMO Soybeans, High Protein, LowCarb, Fitsness, Meal Prep, Handmade
5. Goursmet Instants, Taiwanese Eggnod, Juicy Egg,
--------------------------------------------------


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


inputs355:

1. Freshly Harvested Lala Mountain Peaches, Group Purchase, Limited Time Deal, Sweet and Juicy Peaches, Local Sourced, Summer Delicacy, Aromatic Flavor。

outputs:

1. Freshly Harvested Lala Mountain Peaches, Group Purchase, Limited Time Deal, Sweet and Juicy Peaches, Local Sourced, Summer Delicacy, Aromatic Flavor
2.

 Taiwanese Fresh Chicken, Smoked Chicken Slices, Chicken Breast Selection, Chikoroller Chicken Wings, Chickennibbers, Chiciner Chicken Choices, Tender Juicy Chicken Cuts
3. Frozen Seafood Mix, Squid, Scallops, White Shrimp, Seafoods, Instant Meal Prep, Meal Preparation
4. SmokedChickenPowder, Side Dish, Gourmet Delight, Taiwanese Delicasy, Juicy Smoked Skipped Chicken;
--------------------------------------------------
inputs356:

1. Snack, Appetizer, Smoked, Spicy, Boneless, Sichuan flavor, Ready-to-eat。

outputs:

1. Snack, Appetizer, Smoked, Spicy, Boneless, Sichuan flavor, Ready-to-eat
2.

 Pork trotter, Taiwanese cuisine, Braised pork trotor, Secret recip

In [ ]:
"""MultiBeam Inference"""
max_len = 128 # not sure
generation_configs = [
    GenerationConfig(
        do_sample=True,
        temperature=0.1, # important
        num_beams=1,
        top_p=0.3, # not sure
        no_repeat_ngram_size=1, # important (higher less diversity)
        pad_token_id=2,
    ),
    GenerationConfig(
        do_sample=True,
        temperature=0.1, # important
        num_beams=1,
        top_p=0.3, # not sure
        no_repeat_ngram_size=3, # important (higher less diversity)
        pad_token_id=2,
    ),
    GenerationConfig(
        do_sample=True,
        temperature=0.1, # important
        num_beams=1,
        top_p=0.3, # not sure
        no_repeat_ngram_size=5, # important (higher less diversity)
        pad_token_id=2,
    ),
]
multi_beam_inference_results = {}
for i, pid in enumerate(final.AVAILABLE_PRODUCT_IDS):
    d = inference_dataset[i]
    predicts = llm.multi_beam_inference(d, generation_configs, max_len, verbose=False)
    inputs_ = '\n'.join(d['input'].split('\n')[:-1])
    print(f'inputs:\n{d["instruction"]}\n{inputs_}。\n')
    print(f'outputs:\n2. ' + predicts)
    print('-' * 50)
    multi_beam_inference_results[pid] = predicts

## Results Parsing

Since it's hard for LLM to perfectly follow the given format, the generation may need to manually parsed.

In [2]:
import final
print(final.data.product_tags_v5_en())

{5: ['Love Ataulfo Mango', 'Mango', 'Mango Ice', 'Fresh Mangoes', 'Exquisite Fruits', 'Fruit Gift Box', 'Juicy Mango', 'Premium Mango', 'Summer Delight', 'Perfect Gift'], 23: ['Frozen Broccoli', 'Cruciferous Vegetables', 'Cancer-fighting', 'Low-Calorie Diet', 'Healthy Eating', 'Quick Meal Prep', 'Fitness Recipes', 'Weight Loss', 'Low-Carb Option', 'Salad Ingredient'], 24: ['Frozen Cauliflower', 'Healthy Eating', 'Meal Preparation', 'Fitness', 'Weight Control', 'Quick Meals', 'Frozen Vegetables', 'Nutritious Ingredients', 'Diet Control', "Kids' Lunch"], 25: ['Healthy Eating', 'Frozen Vegetables', 'Meal Prep', 'Fitness Food', 'Health Recipes', 'Controlled Diet', 'Quick Meals', 'Broccoli', 'Cauliflower', 'Convenient'], 28: ['Pork Balls', 'Taiwan Pork', 'Hot Pot Ingredients', 'Fresh Meat', 'Pork Leg', 'Meatballs', 'Pure Pork', 'Yummy Soup', 'Local Ingredients', 'Meat Density'], 46: ['Gourmet Fish Dish', 'Taiwan Sea Bass', 'Post-surgery Diet', 'Meal Prep', 'Sea Bass Recipe', 'Nutritious Mea

In [8]:
import json
import os
assert set(single_inference_results) == set(final.AVAILABLE_PRODUCT_IDS)
log_dir = 'results/ckpt-14-2'
os.makedirs(log_dir, exist_ok=True)
with open(os.path.join(log_dir, 'generation_config.json'), 'w', encoding='utf8') as fout:
    json.dump(generation_config.to_dict(), fout, indent=4)
with open(os.path.join(log_dir, 'raw_predict.json'), 'w', encoding='utf8') as fout:
    json.dump(single_inference_results, fout, ensure_ascii=False)
# single_inference_results

In [ ]:
"""Parse the results here"""

enhanced_tags: dict[int, list[str]] = ...
# keys are pids, values are tags
assert set(enhanced_tags) == set(final.AVAILABLE_PRODUCT_IDS)
"""Save the enhanced tags if the results satisfy !!!!"""

In [ ]:
assert set(enhanced_tags) == set(final.AVAILABLE_PRODUCT_IDS)
import pandas as pd
rows = []
for split, merged_df in final.data.merged_df().items():
    baseline_scores = final.utils.baseline_score(merged_df)
    metrics = final.ranking_metrics(baseline_scores, merged_df['y_true'])
    metrics['name'] = f'baseline/{split}'
    rows.append(metrics)

    metrics = final.TagsEvaluator(PRODUCT_RAW_TAGS).cal_ranking_metrics(merged_df)
    metrics['name'] = f'raw_tags/{split}'
    rows.append(metrics)

    metrics = final.TagsEvaluator(enhanced_tags).cal_ranking_metrics(merged_df)
    metrics['name'] = f'enhanced_tags/{split}'
    rows.append(metrics)

res_df = pd.DataFrame(rows).set_index('name')
print(res_df)